In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os
import math



In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

In [3]:
raw_file="Raw data 03-May'25 According to 4_7.5.csv"
gst_file="GST 03-May'25 According to 4_7.5.csv"
tracker_file="EDD Tracker till 03rd May'25.csv"
format_file="edd_format.csv"
ops_file="ops_status.csv"
#old_tracker_file="old_tracker.csv"


In [4]:
raw_data= pd.read_csv(raw_file,memory_map=True)
gst_data=pd.read_csv(gst_file,memory_map=True)
tracker_data=pd.read_csv(tracker_file,memory_map=True,names=['bank_response', 'final_remarks'], header=0)
master_format=pd.read_csv(format_file,memory_map=True)
ops_status=pd.read_csv(ops_file,memory_map=True,names=['bank_response', 'final_remarks'], header=0)
#old_tracker=pd.read_csv(old_tracker_file,memory_map=True,names=['bank_response', 'final_remarks'], header=0)

In [5]:
raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,85680803,yfNxHL07598084892013,yfNxHL07598084892013,85680803,2024-11-09 18:11:56.000,2024-11-09 18:12:30.000,1.325563e+09,DIY,QR,PTY,18185680803,"{""sellerId"":""18185680803""}",PTYES,MISPK3811K,MISPK3811K,NaN,No,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,27530.0,400000,888.064516,27530.0,1,1,0.068825,Not_required,Pending for share,100000.0
1,84186586,mYhNVn06957843135988,mYhNVn06957843135988,84186586,2024-06-14 18:34:47.000,2024-06-15 11:14:30.000,1.282846e+09,Offline Unorganized,EDC,PTY,02684186586,"{""sellerId"":""02684186586""}",PTYES,GNMPS7558L,GNMPS7558L,NaN,No,Food,Sweet Shop,5812,PROPRIETORSHIP,95032.0,400000,3065.548387,95032.0,1,1,0.237580,Not_required,Pending for share,100000.0
2,85037031,zOGJFj01760622765934,zOGJFj01760622765934,85037031,2024-09-18 10:59:22.000,2024-09-18 11:00:05.000,5.898718e+08,Offline Unorganized,QR,PTY,18785037031,"{""sellerId"":""18785037031""}",PTYES,APHPH3879L,APHPH3879L,09APHPH3879L1Z8,Yes,Wholesale Trade and B2B,Medical Equipment,5047,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [6]:
gst_data=gst_data.rename(columns={'_col3':'updated_at','solution_value.1':'gst'})


# Convert 'date' column to datetime type
gst_data['updated_at'] = pd.to_datetime(gst_data['updated_at'], format='%d-%m-%Y', errors='coerce')

gst_data.head(3)

,solution_value,cust_id,gst,updated_at
0,mVABSi59986777757097,320138362.0,Slab: Rs. 40 lakhs to 1.5 Cr.,NaT
1,NCVWXj99002615035149,440958050.0,Slab: Rs. 5 Cr. to 25 Cr.,NaT
2,qUZsnH88171384595093,571296992.0,Slab: Rs. 0 to 40 lakhs,NaT


In [7]:


# Sort by id and date (latest date first)
gst_data = gst_data.sort_values(by=['solution_value', 'updated_at'], ascending=[True, False])

# Drop duplicates: keep the first (which is the latest date because of sort)
gst_data = gst_data.drop_duplicates(subset='solution_value', keep='first')

# Reset index if needed
gst_data = gst_data.reset_index(drop=True)

In [8]:
# Convert the two columns to datetime first
raw_data['mid_creation_date'] = pd.to_datetime(raw_data['mid_creation_date'], errors='coerce')
raw_data['channel_date'] = pd.to_datetime(raw_data['channel_date'], errors='coerce')

# Then format them to 'dd-mm-yyyy' string format
raw_data['mid_creation_date'] = raw_data['mid_creation_date'].dt.strftime('%d-%m-%Y')
raw_data['channel_date'] = raw_data['channel_date'].dt.strftime('%d-%m-%Y')

raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,85680803,yfNxHL07598084892013,yfNxHL07598084892013,85680803,09-11-2024,09-11-2024,1.325563e+09,DIY,QR,PTY,18185680803,"{""sellerId"":""18185680803""}",PTYES,MISPK3811K,MISPK3811K,NaN,No,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,27530.0,400000,888.064516,27530.0,1,1,0.068825,Not_required,Pending for share,100000.0
1,84186586,mYhNVn06957843135988,mYhNVn06957843135988,84186586,14-06-2024,15-06-2024,1.282846e+09,Offline Unorganized,EDC,PTY,02684186586,"{""sellerId"":""02684186586""}",PTYES,GNMPS7558L,GNMPS7558L,NaN,No,Food,Sweet Shop,5812,PROPRIETORSHIP,95032.0,400000,3065.548387,95032.0,1,1,0.237580,Not_required,Pending for share,100000.0
2,85037031,zOGJFj01760622765934,zOGJFj01760622765934,85037031,18-09-2024,18-09-2024,5.898718e+08,Offline Unorganized,QR,PTY,18785037031,"{""sellerId"":""18785037031""}",PTYES,APHPH3879L,APHPH3879L,09APHPH3879L1Z8,Yes,Wholesale Trade and B2B,Medical Equipment,5047,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [9]:
tracker_data=tracker_data.fillna(0)
tracker_data.isna().sum()

bank_response    0
final_remarks    0
dtype: int64

In [10]:
tracker_data.head(3)

,bank_response,final_remarks
0,"{""sellerId"":""11285953360""}",EDD/ODD approved & Pending at Business Need fi...
1,"{""sellerId"":""19385979804""}",EDD/ODD approved & Pending at Business Need fi...
2,"{""sellerId"":""15486127866""}",EDD/ODD approved & Pending at Business Need fi...


In [11]:
len(ops_status)+len(tracker_data)#+len(old_tracker)

191750

In [12]:
print(tracker_data.columns)
#print(old_tracker.columns)
print(ops_status.columns)

Index(['bank_response', 'final_remarks'], dtype='object')
Index(['bank_response', 'final_remarks'], dtype='object')


In [13]:
#tracker_data=tracker_data.rename(columns={'Bank Response':'bank_response','Final Remarks':'final_remarks'})
#old_tracker=old_tracker.rename(columns={'bank_response':'bank_response','Final Remarks':'final_remarks'})
#ops_status=ops_status.rename(columns={'sellerid':'bank_response','OPS status':'final_remarks'})



ops_status_data= pd.concat([ops_status, tracker_data], axis=0, ignore_index=True)
print('totol tracker data ',len(ops_status_data))
print(ops_status_data.isna().sum())


totol tracker data  191750
bank_response    0
final_remarks    0
dtype: int64


In [ ]:
len(raw_data)

In [14]:
# Make sure everything is a string first
raw_data['sellerid'] = raw_data['sellerid'].astype(str)
raw_data['entity_id'] = raw_data['entity_id'].astype(str)
raw_data['parent_entity_id'] = raw_data['parent_entity_id'].astype(str)

# Calculate the length of each entity_id
raw_data['entity_len'] = raw_data['entity_id'].str.len()

# Extract from seller_id the last N characters based on entity_len
raw_data['seller_tail'] = raw_data.apply(lambda x: x['sellerid'][-int(x['entity_len']):], axis=1)


# Step 3: Compare both
raw_data['is_entity_match'] = raw_data['seller_tail'] == raw_data['entity_id']
raw_data['is_parent_match'] = raw_data['seller_tail'] == raw_data['parent_entity_id']

# Step 4: Create final match column
raw_data['final_match'] = raw_data['is_entity_match'] | raw_data['is_parent_match']

raw_data = raw_data[raw_data['final_match']]


len(raw_data)



284968

In [15]:
master_format[['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank']]=raw_data[['parent_id', 'mid','parent_entity_id','entity_id',
       'mid_creation_date', 'channel_date', 'cust_id', 'channel', 'edc_flag',
       'vpa_type', 'sellerid', 'bank_response', 'bank_name', 'pan_no_business',
       'pan_no_personal', 'gstin', 'gst_flag', 'category', 'sub_category',
       'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag', 'drr',
       'month_expected_gmv', 'total_child_mid', 'rank']]

len(master_format)

284968

In [16]:
gst_mapping = gst_data.set_index('solution_value')['gst'].to_dict()
master_format['GST Turn over'] = master_format['parent_id'].map(gst_mapping)

# Fill missing GSTs with 'NA'
master_format['GST Turn over'] = master_format['GST Turn over'].fillna('NA')

In [17]:
ops_status_mapping = ops_status_data.set_index('bank_response')['final_remarks'].to_dict()
master_format['OPS team status'] = master_format['bank_response'].map(ops_status_mapping)

# Fill missing GSTs with 'NA'
master_format['OPS team status'] = master_format['OPS team status'].fillna('Pending for share')

In [18]:
master_format.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1'],
      dtype='object')

In [19]:

master_format['Proposed Limit'] = np.where(
    master_format['Month_expected_GMV'].isna(), 
    master_format['limit_flag'], 
    np.maximum(np.ceil(master_format['Month_expected_GMV'] / 1e5) * 1e5, master_format['limit_flag'])
)


In [20]:

# First replace NaN in 'gmv' with 0 for calculation
master_format['% usage'] = (master_format['gmv'].fillna(0) / master_format['limit_flag']) * 100




In [21]:

# Make sure % usage is numeric
master_format['% usage'] = pd.to_numeric(master_format['% usage'], errors='coerce')

# Define bins and labels
bins = [-float('inf'), 5, 15, 40, 60, 80, 100, float('inf')]
labels = ['1_<5 %', '2_5 to 15 %', '3_15-40%', '4_40-60%', '5_60-80%', '6_80-100%', '7.above 100%']

# Apply binning
master_format['Grouping'] = pd.cut(master_format['% usage'], bins=bins, labels=labels, right=False)



master_format['Grouping'].value_counts()


Grouping
1_<5 %          151135
7.above 100%     49090
3_15-40%         31704
2_5 to 15 %      27829
4_40-60%         11874
5_60-80%          8113
6_80-100%         5223
Name: count, dtype: int64

In [22]:

# Suppose your DataFrame is named df
liquor_categories = ['Alcoholic Beverages', 'Restaurants and Bars', 'wine and liquor shop', 'Wine producers','Alcoholic beverages']

master_format['Liquor/Others'] = master_format['sub_category'].apply(lambda x: 'Liquor' if x in liquor_categories else 'Others')


master_format['Liquor/Others'].value_counts()



Liquor/Others
Others    272321
Liquor     12647
Name: count, dtype: int64

In [23]:


master_format['Limit Required/Not'] = np.where(master_format['Proposed Limit'] > master_format['limit_flag'], 'Required', 'Not_required')

master_format['Limit Required/Not'].value_counts()


Limit Required/Not
Not_required    225263
Required         59705
Name: count, dtype: int64

In [24]:

master_format['Sum of GMV'] = master_format.groupby('bank_response')['gmv'].transform('sum')

master_format['Sum of DRR'] = master_format.groupby('bank_response')['DRR'].transform('sum')

master_format['Sum of Month Expected GMV'] = master_format.groupby('bank_response')['Month_expected_GMV'].transform('sum')

master_format['Sum of limit_flag'] = master_format.groupby('bank_response')['limit_flag'].transform('sum')

# Use numpy's vectorized operations  (MAX and ROUNDUP equivalent)
master_format['Sum of Proposed Limit'] = np.maximum(np.ceil(master_format['Sum of Month Expected GMV'] / 10**5) * 10**5, master_format['Sum of limit_flag'])

# Use numpy for the IF condition (AO2 > AN2)
master_format['Limit Required/Not.1'] = np.where(master_format['Sum of Proposed Limit'] > master_format['Sum of limit_flag'], "Required", "Not_required")

In [25]:
master_format.head(5)

,parent_id,Child MID,Parent Entity_Id,Child Entity_Id,MID Creation Date,PTY Creation Date,Cust_id,channel,EDC Flag,vpa_type,sellerid,bank_response,bank_name,Pan No Business,Pan No Personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,DRR,Month_expected_GMV,total_Child_mid,rank,Proposed Limit,% usage,Grouping,GST Turn over,Liquor/Others,Limit Required/Not,Business status,OPS team status,YBL status,Sum of GMV,Sum of DRR,Sum of Month Expected GMV,Sum of limit_flag,Sum of Proposed Limit,Limit Required/Not.1
0,yfNxHL07598084892013,yfNxHL07598084892013,85680803,85680803,09-11-2024,09-11-2024,1.325563e+09,DIY,QR,PTY,18185680803,"{""sellerId"":""18185680803""}",PTYES,MISPK3811K,MISPK3811K,NaN,No,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,27530.0,400000,888.064516,27530.0,1,1,400000.0,6.8825,2_5 to 15 %,NA,Others,Not_required,NaN,ODD/EDD closed and no action required (Limit a...,NaN,27530.0,888.064516,27530.0,400000,400000.0,Not_required
1,mYhNVn06957843135988,mYhNVn06957843135988,84186586,84186586,14-06-2024,15-06-2024,1.282846e+09,Offline Unorganized,EDC,PTY,02684186586,"{""sellerId"":""02684186586""}",PTYES,GNMPS7558L,GNMPS7558L,NaN,No,Food,Sweet Shop,5812,PROPRIETORSHIP,95032.0,400000,3065.548387,95032.0,1,1,400000.0,23.7580,3_15-40%,NA,Others,Not_required,NaN,Pending for share,NaN,95032.0,3065.548387,95032.0,400000,400000.0,Not_required
2,zOGJFj01760622765934,zOGJFj01760622765934,85037031,85037031,18-09-2024,18-09-2024,5.898718e+08,Offline Unorganized,QR,PTY,18785037031,"{""sellerId"":""18785037031""}",PTYES,APHPH3879L,APHPH3879L,09APHPH3879L1Z8,Yes,Wholesale Trade and B2B,Medical Equipment,5047,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.0000,1_<5 %,Slab: Rs. 40 lakhs to 1.5 Cr.,Others,Not_required,NaN,ODD/EDD closed and no action required (Limit a...,NaN,0.0,0.000000,0.0,750000,750000.0,Not_required
3,zPSRzX85990739978881,zPSRzX85990739978881,71545155,71545155,10-07-2022,26-03-2025,1.148615e+09,DIY,QR,PTY,15071545155,"{""sellerId"":""15071545155""}",PTYES,ENNPR8508A,ENNPR8508A,NaN,No,Healthcare,Pharmacy,5912,PROPRIETORSHIP,10940.0,400000,437.600000,13565.6,1,1,400000.0,2.7350,1_<5 %,NA,Others,Not_required,NaN,ODD/EDD closed and no action required (Limit a...,NaN,10940.0,437.600000,13565.6,400000,400000.0,Not_required
4,zHkWwA99081997626879,zHkWwA99081997626879,84937045,84937045,09-09-2024,09-09-2024,4.449731e+06,Offline Unorganized,EDC,PTY,13284937045,"{""sellerId"":""13284937045""}",PTYES,BKHPB0327D,BKHPB0327D,21BKHPB0327D1Z5,Yes,CDIT,Electronics and Home Appliances,5722,PROPRIETORSHIP,6450.0,750000,1290.000000,39990.0,1,1,750000.0,0.8600,1_<5 %,Slab: Rs. 0 to 40 lakhs,Others,Not_required,NaN,ODD/EDD closed and no action required (Limit a...,NaN,6450.0,1290.000000,39990.0,750000,750000.0,Not_required


In [26]:
#master_format.to_csv('master_format.csv',index=False)

In [27]:
pending_data =master_format[master_format['OPS team status']=='Pending for share']
len(pending_data)

46531

In [28]:
pending_data['category'].unique()

array(['Food', 'Retail and Shopping', 'Wholesale Trade and B2B', 'Travel',
       'Individual Services', 'CDIT', 'Agriculture',
       'Automobiles and Vehicles', 'Milk Dairy and Cooperative',
       'LFR and Brands', 'Beauty and Wellness', 'Healthcare',
       'Personal Services', 'NGO', 'Business Services', 'Logistics',
       'Entertainment', 'Devotion', 'Event Services', 'Grocery',
       'Corporate Alliance', 'Home Services', 'GROCERY', 'Government',
       'Utilities', 'Tuition and Classes', 'Education',
       'Cable and Broadband', 'Airport', 'Housing', 'Gas and Petrol',
       'Digital Goods', 'Hotel', 'Transport', 'Tolls',
       'Parking and Tolls', 'GAS PETROL', 'BFSI', 'Parking'], dtype=object)

In [ ]:
# Create a new column 'remark'
pending_data['Remark'] = ''

# Set remark as 'share' where cat is 'Gas and Petrol' or 'Gas Petrol'
pending_data.loc[pending_data['category'].isin(['Gas and Petrol','GAS PETROL' ,'Gas Petrol']), 'Remark'] = 'O&G'

C:\Users\avnish1.yadav\AppData\Local\Temp\ipykernel_20812\2633286974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pending_data['Remark'] = ''


In [31]:

special_cat = {
    'Category': [
        'Retail and Shopping', 'Food', 'Retail and Shopping', 'Wholesale Trade and B2B',
        'Healthcare', 'Healthcare', 'Education', 'Education', 'Education', 'Education',
        'Healthcare', 'Food','Retail and Shopping'
    ],
    'Subcategory': [
        'wine and liquor shop', 'Restaurants and Bars', 'Alcoholic Beverages', 'Wine producers',
        'Hospital', 'Pharmacy', 'School - Secondary & Higher Secondary', 'School - Primary',
        'School', 'Private College', 'Medical College', 'Food','Alcoholic beverages'
    ],
    'Main Group': [
        'Liquor', 'Liquor', 'Liquor', 'Liquor',
        'Hospital', 'Pharmacy', 'Education', 'Education',
        'Education', 'Education', 'Education', 'Food','Liquor'
    ]
}

special_category = pd.DataFrame(special_cat)

In [32]:
pending_data.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1', 'Remark'],
      dtype='object')

In [33]:

# Perform a left merge to get the corresponding Main Group based on cat and sub_cat
pending_data = pd.merge(pending_data, special_category, how='left', left_on=['category', 'sub_category'], right_on=['Category', 'Subcategory'])

# Update the 'remark' column only where there is a match


pending_data['Remark'] = pending_data.apply(
    lambda row: row['Main Group'] if pd.notna(row['Main Group']) else row['Remark'],
    axis=1
)


pending_data = pending_data.drop(['Category', 'Subcategory','Main Group'], axis=1)

In [34]:
pending_data['Remark'].value_counts()

Remark
             46376
O&G             67
Liquor          50
Pharmacy        32
Hospital         5
Education        1
Name: count, dtype: int64

In [35]:
pending_data['bank_response_count'] = pending_data['bank_response'].map(pending_data['bank_response'].value_counts())

In [36]:
turn_over = {
    'GST Return Slab': [
        'without GSTIN only pan With Business Proof',
        'GSTIN Turnover NA',
        'Slab: Rs. 0 to 40 lakhs',
        'Slab: Rs. 40 lakhs to 1.5 Cr.',
        'Slab: Rs. 1.5 Cr. to 5 Cr.',
        'Slab: Rs. 5 Cr. to 25 Cr.',
        'Slab: Rs. 25 Cr. to 100 Cr.',
        'Slab: Rs. 100 Cr. to 500 Cr.',
        'Slab: Rs. 500 Cr. and above',
        'Slab: Rs. 25 Cr. to 100 Cr.',
        'Slab: Rs.0 to 40 lakhs',
        'Slab: Rs.1.5 Cr. to 5 Cr.',
        'Slab: Rs.25 Cr. to 100 Cr.',
        'Slab: Rs.40 lakhs to 1.5 Cr.',
        'Slab: Rs.5 Cr. to 25 Cr.',
        'NA'
    ],
    'Limit as per Slab': [
        600000,
        750000,
        750000,
        750000,
        2500000,
        6000000,
        10000000,
        10000000,
        10000000,
        10000000,
        750000,
        2500000,
        10000000,
        750000,
        6000000,
        600000
    ]
}

gst_turn_over = pd.DataFrame(turn_over)

In [37]:
# Remove duplicates for clean mapping
gst_turn_over = gst_turn_over.drop_duplicates('GST Return Slab')

# Create mapping dictionary
slab_to_limit = dict(zip(gst_turn_over['GST Return Slab'], gst_turn_over['Limit as per Slab']))


# Map the limits based on slab
pending_data['Limit as per Slab'] = pending_data['GST Turn over'].map(slab_to_limit)


In [38]:
pending_data.tail(3)

,parent_id,Child MID,Parent Entity_Id,Child Entity_Id,MID Creation Date,PTY Creation Date,Cust_id,channel,EDC Flag,vpa_type,sellerid,bank_response,bank_name,Pan No Business,Pan No Personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,DRR,Month_expected_GMV,total_Child_mid,rank,Proposed Limit,% usage,Grouping,GST Turn over,Liquor/Others,Limit Required/Not,Business status,OPS team status,YBL status,Sum of GMV,Sum of DRR,Sum of Month Expected GMV,Sum of limit_flag,Sum of Proposed Limit,Limit Required/Not.1,Remark,bank_response_count,Limit as per Slab
46528,PZXGIH28408168318614,PZXGIH28408168318614,85083146,85083146,21-09-2024,21-09-2024,1.774574e+09,Offline Unorganized,EDC,PTY,14085083146,"{""sellerId"":""14085083146""}",PTYES,DROPM5406G,DROPM5406G,NaN,No,Beauty and Wellness,Salon and Parlour,7230,PROPRIETORSHIP,37055.0,400000,1323.392857,41025.178571,1,1,400000.0,9.26375,2_5 to 15 %,NA,Others,Not_required,NaN,Pending for share,NaN,37055.0,1323.392857,41025.178571,400000,400000.0,Not_required,,1,600000
46529,PrtSbV99638398285014,PrtSbV99638398285014,83925378,83925378,10-05-2024,13-05-2024,1.899914e+09,Offline Unorganized,EDC,PTY,17383925378,"{""sellerId"":""17383925378""}",PTYES,AAECB6735E,AAECB6735E,06AAECB6735E1ZV,Yes,Travel,Hotel,7011,PRIVATE_LIMITED,NaN,750000,NaN,NaN,1,1,750000.0,0.00000,1_<5 %,NA,Others,Not_required,NaN,Pending for share,NaN,0.0,0.000000,0.000000,750000,750000.0,Not_required,,1,600000
46530,LLRZPL91489232658845,LLRZPL91489232658845,86328248,86328248,28-12-2024,28-12-2024,2.378205e+08,Offline Unorganized,EDC,PTY,16586328248,"{""sellerId"":""16586328248""}",PTYES,AGYPJ4654M,AGYPJ4654M,08AGYPJ4654M1ZH,Yes,Retail and Shopping,Electrical Shop,5065,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.00000,1_<5 %,Slab: Rs. 0 to 40 lakhs,Others,Not_required,NaN,Pending for share,NaN,0.0,0.000000,0.000000,750000,750000.0,Not_required,,1,750000


In [39]:
pending_data.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1', 'Remark',
       'bank_response_count', 'Limit as per Slab'],
      dtype='object')

In [40]:
pending_data['Remark'].value_counts()

Remark
             46376
O&G             67
Liquor          50
Pharmacy        32
Hospital         5
Education        1
Name: count, dtype: int64

In [41]:
# Apply the condition and update remark
condition = (
    (pending_data['Remark'].isna() | (pending_data['Remark'].str.strip() == '')) &
    (pending_data['gst_flag'].str.lower() == 'yes') &
    (pending_data['Sum of Proposed Limit'] > 750000)
)

pending_data.loc[condition, 'Remark'] = 'GST = YES & Sum of Proposed Limit > 750000'


In [42]:
pending_data['Remark'].value_counts()

Remark
                                              46352
O&G                                              67
Liquor                                           50
Pharmacy                                         32
GST = YES & Sum of Proposed Limit > 750000       24
Hospital                                          5
Education                                         1
Name: count, dtype: int64

In [43]:
# Apply the condition and update remark
condition = (
    (pending_data['Remark'].isna() | (pending_data['Remark'].str.strip() == '')) &
    (pending_data['gst_flag'].str.lower() == 'no') &
    (pending_data['Sum of Proposed Limit'] > 400000)
)

pending_data.loc[condition, 'Remark'] = 'GST = NO & Sum of Proposed Limit > 400000'

In [44]:
pending_data['Remark'].value_counts()

Remark
                                              46325
O&G                                              67
Liquor                                           50
Pharmacy                                         32
GST = NO & Sum of Proposed Limit > 400000        27
GST = YES & Sum of Proposed Limit > 750000       24
Hospital                                          5
Education                                         1
Name: count, dtype: int64

In [45]:
# Apply the condition and update remark
condition = (
    (pending_data['Remark'].isna() | (pending_data['Remark'].str.strip() == '')) &
    (pending_data['gst_flag'].str.lower() == 'yes') &
    ((pending_data['Sum of Proposed Limit'] - pending_data['Limit as per Slab'])< 0)
)

pending_data.loc[condition, 'Remark'] = 'GST = YES & (Proposed Limit < GST Slab Limit)'

In [46]:
pending_data['Remark'].value_counts()

Remark
                                                 46287
O&G                                                 67
Liquor                                              50
GST = YES & (Proposed Limit < GST Slab Limit)       38
Pharmacy                                            32
GST = NO & Sum of Proposed Limit > 400000           27
GST = YES & Sum of Proposed Limit > 750000          24
Hospital                                             5
Education                                            1
Name: count, dtype: int64

In [47]:
master_format.to_csv('master_format.csv',index=False)

pending_data.to_csv('pending_data.csv',index=False)